In [8]:
import os
from dotenv import load_dotenv
import pinecone # Pinecone 사용을 위해 추가
from pathlib import Path
import json

# pip install -r requirements.txt

# .env 파일 로드
load_dotenv()

# OpenAI API 키 (환경변수에서 로드)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# --- Pinecone 설정 추가 ---
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")
INDEX_NAME = os.getenv("INDEX_NAME", "ddauction-db")
# -------------------------

# 현재 코드에서 사용되지 않는 변수들은 그대로 유지
IMAGE_DIR = os.getenv('IMAGE_DIR', 'image_jpg')

# API 키 및 Pinecone 설정 검증
if not OPENAI_API_KEY:
    print("⚠️  OPENAI_API_KEY가 설정되지 않았습니다. .env 파일 또는 환경변수를 확인하세요.")

if not PINECONE_API_KEY or not PINECONE_ENVIRONMENT:
    print("⚠️  PINECONE_API_KEY 또는 PINECONE_ENVIRONMENT가 설정되지 않았습니다. .env 파일을 확인하세요.")


In [10]:
import fitz  # PyMuPDF
from pathlib import Path

PDF_DIR = Path("./pdfs")
MD_DIR = Path("./rag-api/docs")
MD_DIR.mkdir(parents=True, exist_ok=True)

def pdf_to_md(pdf_path: Path, md_path: Path):
    doc = None
    try:
        doc = fitz.open(pdf_path)
        with open(md_path, "w", encoding="utf-8") as f:
            for page in doc:
                text = page.get_text("text")
                f.write(f"\n\n## Page {page.number + 1}\n")
                f.write(text.strip() + "\n")
        print(f"✅ {pdf_path.name} -> {md_path.name}")
    except Exception as e:
        print(f"❌ 오류: {pdf_path.name} : {e}")
    finally:
        if doc:
            doc.close()

if __name__ == "__main__":
    for pdf_file in PDF_DIR.glob("*.pdf"):
        md_file = MD_DIR / (pdf_file.stem + ".md")  # PDF 이름 그대로 .md
        pdf_to_md(pdf_file, md_file)

    print("모든 PDF 변환 완료!")


✅ auction_policy.pdf -> auction_policy.md
✅ general_service_policy.pdf -> general_service_policy.md
✅ prohibited_items_policy.pdf -> prohibited_items_policy.md
✅ sales_and_returns_faq.pdf -> sales_and_returns_faq.md
✅ shipping_delivery_policy.pdf -> shipping_delivery_policy.md
✅ used_item_guidelines.pdf -> used_item_guidelines.md
모든 PDF 변환 완료!
